# 🔨 Build APK per Comic Viewer
Esegui le celle in ordine per compilare l'APK Android

In [ ]:
# 1. Installa dipendenze (incluso libtool per libffi)
!pip install buildozer cython

# Installa libtool, autoconf-archive e altre dipendenze necessarie
!sudo apt-get update
!sudo apt-get install -y \
    python3-pip build-essential git python3-dev ffmpeg \
    libsdl2-dev libsdl2-image-dev libsdl2-mixer-dev libsdl2-ttf-dev \
    libportmidi-dev libswscale-dev libavformat-dev libavcodec-dev zlib1g-dev \
    libgstreamer1.0-dev gstreamer1.0-plugins-base gstreamer1.0-plugins-good \
    libgstreamer-plugins-base1.0-dev libgstreamer-plugins-good1.0-dev \
    libtool libtool-bin autoconf autoconf-archive automake \
    pkg-config libffi-dev cmake ninja-build \
    unzip zip wget curl lld

In [ ]:
# 2. Installa Java e configura l'ambiente
!sudo apt-get install -y openjdk-17-jdk

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}"

# Verifica Java
!java -version

# Pulisci cache buildozer precedenti (se esistono)
!rm -rf ~/.buildozer
!rm -rf /content/comic_viewer/.buildozer

In [ ]:
# 3. Crea cartella progetto
!mkdir -p /content/comic_viewer
%cd /content/comic_viewer

In [ ]:
%%writefile main.py
import os
import requests
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.image import AsyncImage
from kivy.uix.button import Button
from kivy.uix.label import Label
from kivy.uix.popup import Popup
from kivy.uix.spinner import Spinner
from kivy.uix.scatter import Scatter
from kivy.core.window import Window
from kivy.uix.screenmanager import ScreenManager, Screen
from kivy.properties import StringProperty, NumericProperty
from kivy.clock import Clock
from kivy.graphics import Color, Rectangle

# --- CONFIGURAZIONE ---
GITHUB_REPO = "sagara939/Visualizzatore_vignette"
GITHUB_API_BASE = f"https://api.github.com/repos/{GITHUB_REPO}/contents"
GITHUB_RAW_BASE = f"https://raw.githubusercontent.com/{GITHUB_REPO}/main"

IMAGE_EXTENSIONS = ['.png', '.jpg', '.jpeg', '.gif', '.webp']

SERIES_CONFIG = {
    "Serie 1": "comics/serie1",
}


class ComicImage(Scatter):
    source = StringProperty("")
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.do_rotation = False
        self.do_translation = True
        self.do_scale = True
        self.scale_min = 1.0
        self.scale_max = 5.0
        
        self.image = AsyncImage(
            source=self.source,
            fit_mode='contain',
            size_hint=(1, 1),
            pos_hint={'center_x': 0.5, 'center_y': 0.5}
        )
        self.bind(source=self._update_source)
        self.bind(size=self._update_image_size)
        self.add_widget(self.image)
    
    def _update_source(self, instance, value):
        self.image.source = value
        self.scale = 1.0
        self.pos = (0, 0)
    
    def _update_image_size(self, instance, value):
        self.image.size = self.size


class ComicScreen(Screen):
    current_series = StringProperty("")
    current_index = NumericProperty(0)
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.images_list = []
        self.touch_start_x = 0
        self.swipe_threshold = 100
        self.is_swiping = False
        
        self.layout = BoxLayout(orientation='vertical')
        
        # Header
        self.header = BoxLayout(size_hint_y=0.08, padding=5, spacing=5)
        with self.header.canvas.before:
            Color(0.2, 0.2, 0.2, 1)
            self.header_rect = Rectangle(size=self.header.size, pos=self.header.pos)
        self.header.bind(size=self._update_header_rect, pos=self._update_header_rect)
        
        self.series_spinner = Spinner(
            text='Seleziona Serie',
            values=list(SERIES_CONFIG.keys()),
            size_hint_x=0.5,
            background_color=(0.3, 0.3, 0.8, 1)
        )
        self.series_spinner.bind(text=self.on_series_change)
        
        self.counter_label = Label(text="0/0", size_hint_x=0.35, font_size='14sp')
        
        self.refresh_btn = Button(text="⟳", size_hint_x=0.15, background_color=(0.3, 0.6, 0.3, 1))
        self.refresh_btn.bind(on_press=self.refresh_images)
        
        self.header.add_widget(self.series_spinner)
        self.header.add_widget(self.counter_label)
        self.header.add_widget(self.refresh_btn)
        
        # Image area
        self.image_area = BoxLayout(orientation='vertical', size_hint_y=0.84)
        self.comic_image = ComicImage(size_hint=(1, 1))
        self.image_area.add_widget(self.comic_image)
        
        # Footer
        self.footer = BoxLayout(size_hint_y=0.08, padding=5, spacing=10)
        with self.footer.canvas.before:
            Color(0.2, 0.2, 0.2, 1)
            self.footer_rect = Rectangle(size=self.footer.size, pos=self.footer.pos)
        self.footer.bind(size=self._update_footer_rect, pos=self._update_footer_rect)
        
        self.prev_btn = Button(text="◀ Precedente", background_color=(0.4, 0.4, 0.6, 1))
        self.prev_btn.bind(on_press=self.prev_comic)
        
        self.next_btn = Button(text="Successivo ▶", background_color=(0.4, 0.4, 0.6, 1))
        self.next_btn.bind(on_press=self.next_comic)
        
        self.footer.add_widget(self.prev_btn)
        self.footer.add_widget(self.next_btn)
        
        self.layout.add_widget(self.header)
        self.layout.add_widget(self.image_area)
        self.layout.add_widget(self.footer)
        self.add_widget(self.layout)
        
        Clock.schedule_once(self.init_app, 0.5)
    
    def _update_header_rect(self, instance, value):
        self.header_rect.pos = instance.pos
        self.header_rect.size = instance.size
    
    def _update_footer_rect(self, instance, value):
        self.footer_rect.pos = instance.pos
        self.footer_rect.size = instance.size
    
    def show_status(self, message, duration=2):
        popup = Popup(title='Info', content=Label(text=message), size_hint=(0.8, 0.2), auto_dismiss=True)
        popup.open()
        Clock.schedule_once(lambda dt: popup.dismiss(), duration)
    
    def init_app(self, dt):
        if self.series_spinner.values:
            self.series_spinner.text = self.series_spinner.values[0]
    
    def on_series_change(self, spinner, text):
        if text in SERIES_CONFIG:
            self.current_series = text
            self.current_index = 0
            self.images_list = []
            self.load_images_list()
    
    def load_images_list(self):
        if not self.current_series:
            return
        folder = SERIES_CONFIG[self.current_series]
        api_url = f"{GITHUB_API_BASE}/{folder}"
        self.show_status("Caricamento...")
        try:
            response = requests.get(api_url, timeout=10, headers={'User-Agent': 'ComicViewer-App'})
            if response.status_code == 200:
                files = response.json()
                self.images_list = sorted([f['name'] for f in files if f['type'] == 'file' and any(f['name'].lower().endswith(ext) for ext in IMAGE_EXTENSIONS)])
                if self.images_list:
                    self.current_index = 0
                    self.load_comic()
                else:
                    self.show_status("Nessuna immagine trovata")
                    self.counter_label.text = "0/0"
            else:
                self.show_status(f"Errore: {response.status_code}")
        except Exception as e:
            self.show_status("Errore connessione")
    
    def load_comic(self):
        if not self.images_list:
            return
        if 0 <= self.current_index < len(self.images_list):
            folder = SERIES_CONFIG[self.current_series]
            filename = self.images_list[self.current_index]
            image_url = f"{GITHUB_RAW_BASE}/{folder}/{filename}"
            self.comic_image.source = image_url
            self.counter_label.text = f"{self.current_index + 1}/{len(self.images_list)}"
            self.comic_image.scale = 1.0
            self.comic_image.pos = (0, 0)
    
    def next_comic(self, instance=None):
        if self.images_list and self.current_index < len(self.images_list) - 1:
            self.current_index += 1
            self.load_comic()
        else:
            self.show_status("Ultima immagine")
    
    def prev_comic(self, instance=None):
        if self.images_list and self.current_index > 0:
            self.current_index -= 1
            self.load_comic()
        else:
            self.show_status("Prima immagine")
    
    def refresh_images(self, instance=None):
        self.show_status("Aggiornamento...", 1)
        Clock.schedule_once(lambda dt: self.load_images_list(), 0.5)
    
    def on_touch_down(self, touch):
        self.touch_start_x = touch.x
        self.is_swiping = False
        return super().on_touch_down(touch)
    
    def on_touch_move(self, touch):
        if abs(touch.x - self.touch_start_x) > 50:
            self.is_swiping = True
        return super().on_touch_move(touch)
    
    def on_touch_up(self, touch):
        diff = touch.x - self.touch_start_x
        if self.is_swiping and abs(diff) > self.swipe_threshold:
            if diff > 0:
                self.prev_comic()
            else:
                self.next_comic()
            self.is_swiping = False
            return True
        return super().on_touch_up(touch)


class ComicViewerApp(App):
    def build(self):
        self.title = "Comic Viewer"
        Window.clearcolor = (0.1, 0.1, 0.1, 1)
        sm = ScreenManager()
        sm.add_widget(ComicScreen(name='comic'))
        return sm
    
    def on_pause(self):
        return True
    
    def on_resume(self):
        pass


if __name__ == '__main__':
    ComicViewerApp().run()

In [ ]:
%%writefile buildozer.spec
[app]
title = Comic Viewer
package.name = comicviewer
package.domain = org.sagara939
source.dir = .
source.include_exts = py,png,jpg,kv,atlas
version = 1.0.0

# Requirements - hostpython3 e libffi aggiunti esplicitamente
requirements = python3,kivy,requests,urllib3,charset-normalizer,idna,certifi,hostpython3,libffi

orientation = portrait
fullscreen = 0

# Permessi Android
android.permissions = INTERNET,ACCESS_NETWORK_STATE

# API levels
android.api = 33
android.minapi = 21
android.ndk_api = 21

# Architettura - usa armeabi-v7a che è più compatibile
android.archs = armeabi-v7a

android.allow_backup = True

# Bootstrap SDL2
p4a.bootstrap = sdl2

# Skip alcune recipe problematiche
p4a.local_recipes = 

[buildozer]
log_level = 2
warn_on_root = 1

In [ ]:
# 6. Compila APK (ci vogliono 15-25 minuti la prima volta)
# Prima puliamo eventuali build precedenti fallite
!buildozer android clean
!buildozer android debug 2>&1 | tee build.log

In [ ]:
# 7. Scarica l'APK
from google.colab import files
import glob

apk_files = glob.glob('/content/comic_viewer/bin/*.apk')
if apk_files:
    files.download(apk_files[0])
    print(f"✅ APK pronto: {apk_files[0]}")
else:
    print("❌ APK non trovato. Controlla i log sopra per errori.")